In [ ]:
!pip install PyTDC
!pip install rdkit
!pip install PyBioMed

  Using cached PyBioMed-1.0.tar.gz (11.2 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for PyBioMed: filename=PyBioMed-1.0-py3-none-any.whl size=11266895 sha256=6ccf64174f257dd5fb29c7ef92dbf7c222fc175c94c6bbfcac8b2c85181a8358
  Stored in directory: /root/.cache/pip/wheels/03/c8/be/74f7e3ad04bbbb4a2977551bc19e784919a025b97ba6da1a17
Successfully built PyBioMed


In [ ]:
import pandas as pd
import numpy as np
from tdc.multi_pred import DTI

In [ ]:
data = DTI(name = 'BindingDB_Kd')
# data = DTI(name = 'BindingDB_IC50')
# data = DTI(name = 'BindingDB_Ki')
split = data.get_split()

Downloading...
100%|██████████| 54.4M/54.4M [00:03<00:00, 15.8MiB/s]
Loading...
Done!


In [ ]:
data.convert_to_log(form = 'binding')
# convert back: data.convert_from_log(form = 'binding')

To log space...


In [ ]:
data.harmonize_affinities(mode = 'max_affinity')
# data.harmonize_affinities(mode = 'mean')

The scale is converted to log scale, so we will take the maximum!
The original data has been updated!


,Drug_ID,Drug,Target_ID,Target,Y
0,51.0,O=C(O)CCC(=O)C(=O)O,Q9GZT9,MANDSGGPGGPSPSERDRQYCELCGKMENLLRCSRCRSSFYCCKEH...,6.045709
1,187.0,CC(=O)OCC[N+](C)(C)C,P11229,MNTSAPPAVSPNITVLAPGKGPWQVAFIGITTGLLSLATVTGNLLV...,4.759998
2,187.0,CC(=O)OCC[N+](C)(C)C,P58154,MRRNIFCLACLWIVQACLSLDRADILYNIRQTSRPDVIPTQRDRPV...,6.339040
3,237.0,CCN(CC)CCCC(C)Nc1c2ccc(Cl)cc2nc2ccc(OC)cc12,P02752,MLRFAITLFAVITSSTCQQYGCLEGDTHKANPSPEPNMHECTLYSE...,6.578232
4,242.0,O=C([O-])c1ccccc1,P14920,MRVVVIGAGVIGLSTALCIHERYHSVLQPLDIKVYADRFTPLTTTD...,5.657558
...,...,...,...,...,...
42231,138805907.0,COc1nc2ccc([C@@](O)(c3ccc(C(F)(F)F)nc3)c3cncn3...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,8.494850
42232,138805908.0,COc1nc2ccc([C@](O)(c3ccnc(C(F)(F)F)c3)c3cncn3C...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,5.031512
42233,138805909.0,COc1nc2ccc([C@@](O)(c3ccnc(C(F)(F)F)c3)c3cncn3...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,8.443697
42234,138805910.0,COc1nc2ccc([C@](O)(c3ccc(Cl)cc3)c3cncn3C)cc2c(...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,6.070530


In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Feature extraction for SMILES (drug) strings
def extract_drug_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [0] * 200  # Return a vector of zeros if SMILES is invalid
    features = []
    features.append(Descriptors.MolWt(mol))
    features.append(Descriptors.TPSA(mol))
    features.append(Crippen.MolLogP(mol))
    features.append(Descriptors.NumHAcceptors(mol))
    features.append(Descriptors.NumHDonors(mol))
    # Add additional molecular features if needed
    return features

# Feature extraction for protein sequences
def extract_target_features(sequence):
    # Basic feature extraction example: using amino acid composition
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    aa_counts = [sequence.count(aa) / len(sequence) for aa in amino_acids]
    return aa_counts

# Combine drug and target features
def extract_features(drug, target):
    drug_features = extract_drug_features(drug)
    target_features = extract_target_features(target)
    return drug_features + target_features



Extracting Morgan Fingerprint from Drug SMILES string and CTD descriptors from target protein sequence.

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors, AllChem
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from PyBioMed.PyProtein import CTD

# Feature extraction for SMILES (drug) strings
def extract_drug_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [0] * 2048  # Return a vector of zeros if SMILES is invalid
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    features = list(fingerprint)
    return features

# Feature extraction for protein sequences using CTD descriptors
def extract_target_features(sequence):
    try:
        # Calculate CTD descriptors using PyBioMed
        descriptors = CTD.CalculateCTD(sequence)
        # Extract values as a list to return as feature vector
        feature_vector = list(descriptors.values())
    except Exception as e:
        print(f"Error calculating CTD for sequence: {sequence}")
        feature_vector = [0] * 147
    return feature_vector

# Combine drug and target features
def extract_features(drug, target):
    drug_features = extract_drug_features(drug)
    target_features = extract_target_features(target)
    return drug_features + target_features


In [ ]:
# Load the data
data = data.get_data()

# Prepare the data for training and testing
X = [extract_features(row['Drug'], row['Target']) for _, row in data.iterrows()]
y = data['Y']



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Train a regression model (Random Forest Regressor in this example)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Example prediction for a new SMILES and protein sequence
new_drug_smiles = "CCCCOC(=O)C1=CC=CC=C1"
new_target_sequence = "MRVVVIGAGVIGLSTALCIHERYHSVLQPLDIKVYADRFTPLTTTD"

new_features = extract_features(new_drug_smiles, new_target_sequence)
new_features = scaler.transform([new_features])
predicted_affinity = model.predict(new_features)[0]
print(f"Predicted binding affinity for drug-target pair: {predicted_affinity}")

Mean Squared Error: 0.687417783107528
Predicted binding affinity for drug-target pair: 5.104362279763631


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Define the parameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    # 'min_samples_split': [2, 5, 10],
    # 'min_samples_leaf': [1, 2, 4]
}

# Create a Random Forest Regressor object
rf_regressor = RandomForestRegressor(random_state=42)

# Create a GridSearchCV object
grid_search = GridSearchCV(
    estimator=rf_regressor,
    param_grid=param_grid,
    scoring=['neg_mean_squared_error', 'r2', 'neg_mean_absolute_error'],
    refit='neg_mean_squared_error',  # Refit the model with the best MSE
    cv=5,  # 5-fold cross-validation
    verbose=2  # Print detailed information about the search
)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Negative MSE: {best_score}")

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

# Evaluate the model using different metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")
print(f"Mean Absolute Error: {mae}")

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ....................max_depth=None, n_estimators=50; total time=  25.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=  27.5s
[CV] END ....................max_depth=None, n_estimators=50; total time=  29.5s
[CV] END ....................max_depth=None, n_estimators=50; total time=  26.9s
[CV] END ....................max_depth=None, n_estimators=50; total time=  27.7s
[CV] END ...................max_depth=None, n_estimators=100; total time=  54.8s
[CV] END ...................max_depth=None, n_estimators=100; total time=  54.5s
[CV] END ...................max_depth=None, n_estimators=100; total time=  59.4s
[CV] END ...................max_depth=None, n_estimators=100; total time=  55.4s
[CV] END ...................max_depth=None, n_estimators=100; total time=  52.1s
[CV] END ...................max_depth=None, n_estimators=200; total time= 1.9min
[CV] END ...................max_depth=None, n_est